## Random Forest on Census Data

This code works on classifying a common UCI ML data set, Adults, which attempts to predict whether a person will make over 50k/year based on various characteristics (Age, race, sex, occupation, etc...)

It's a 1994 data set, so conclusions might not be super interesting to today's world, but it's good practice in implementing a random forest.

### Import data

In [37]:
import pandas as pd
import numpy as np

In [38]:
adults = pd.read_csv('adult.csv')
adults

,Age,workclass,FinalWeight,Education,EdNum,MaritalStatus,Occupation,Relationship,Race,Sex,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


### Build a Decision Tree from the data

To build a decision tree, we need to decide which variable to put at the top, and how to split it below, on and on and on. 

We will use the Gini Impurtiy to find which factors are the best to incorporate!

For each column of interest, we will find the Gini impurity. For columns with numeric data or more than 2 options (most columns), we find the gini impurity for each individual value, then pick the lowest gini impurity from that batch to define the gini impurity of the column as a whole.

Let's find the gini impurity of two columns, one categorial and one numerical, to get an idea for the process. Then we'll write up functions to apply that process to other columns.

#### Age--Gini Impurity

The first step in finding the Gini Impurity for this column is ordering the data numerically and finding the mean value in between each value in the column.

In [39]:
#First, we create a numpy array from the dataset and sort it numerically. We also eliminate repeat values (of which there are many, of course)
age = np.unique(np.sort(np.array(adults['Age'])))

#We also print the array to make sure it makes sense.
print(age)

[17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40
 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64
 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88
 90]


We now need to find the average in between each age value. Just looking at this, it's clear it's just the 0.5 increments between the values, but we won't hard code it so it can apply to other variables as well. There might be a numpy package that can do this, but I think it's useful to write it out

In [40]:
age_avg_btwn = np.zeros(len(age) - 1)
for i in range(len(age) - 1):
    age_avg_btwn[i] = (age[i] + age[i+1])/2
print(age_avg_btwn)

[17.5 18.5 19.5 20.5 21.5 22.5 23.5 24.5 25.5 26.5 27.5 28.5 29.5 30.5
 31.5 32.5 33.5 34.5 35.5 36.5 37.5 38.5 39.5 40.5 41.5 42.5 43.5 44.5
 45.5 46.5 47.5 48.5 49.5 50.5 51.5 52.5 53.5 54.5 55.5 56.5 57.5 58.5
 59.5 60.5 61.5 62.5 63.5 64.5 65.5 66.5 67.5 68.5 69.5 70.5 71.5 72.5
 73.5 74.5 75.5 76.5 77.5 78.5 79.5 80.5 81.5 82.5 83.5 84.5 85.5 86.5
 87.5 89. ]


Perfect!

Now we need to calculate the gini impurity for each of these values. Essentially, we will split the data into people younger than and older than each age in this array, and then see how many people in each category make less than 50k or more than 50k. We find the lowest gini inequality for each age, and the age with the lowest will be the representative for the age column.

Gini Impurity = 1 - $(Pincome >50k)^2 - (P
income <50k)^2$

In [41]:
Gini_age = np.zeros(len(age_avg_btwn))
age_salary = np.array(adults[['Age', 'Salary']])
datapoints = len(age_salary)

below_age_50plus = 0
below_age_50minus = 0
above_age_50plus = 0
above_age_50minus = 0


# Loop over each avg age
for count, age_avg_btwn in enumerate(age_avg_btwn):

    # Find the number of people in each category
    for j in age_salary:
        if j[0] < age_avg_btwn:
            if j[1] == '>50K':
                below_age_50plus += 1
            else:
                below_age_50minus += 1
        if j[0] > age_avg_btwn:
            if j[1] == '>50K':
                above_age_50plus += 1
            else:
                above_age_50minus += 1

    #Compute the gini impurity for each leaf

    gini_below_age = 1 - (below_age_50plus/datapoints)**2 - (below_age_50minus/datapoints)**2
    gini_above_age = 1 - (above_age_50plus/datapoints)**2 - (above_age_50minus/datapoints)**2

    #Compute the total gini for this age by weighing the gini impurity 

    total_gini = ((below_age_50minus + below_age_50plus)/datapoints)*gini_below_age + ((above_age_50minus + above_age_50plus)/datapoints)*gini_above_age

    Gini_age[count] = total_gini

    below_age_50minus = 0
    below_age_50plus = 0
    above_age_50minus = 0
    above_age_50plus = 0


Now we have an array, Gini_age, with the gini impurity for each age in age_avg_btwn. We take the age with the lowest gini impurity as the ideal age, and store it in a tuple with its gini impurity, which will be compared with the gini impurities of other values in the future.

In [42]:
#We redfine age_avg_btwn because it was changed in the enumerate function
age_avg_btwn = np.zeros(len(age) - 1)
for i in range(len(age) - 1):
    age_avg_btwn[i] = (age[i] + age[i+1])/2

# Now make the tuple
index_of_min = np.argmin(Gini_age)
Ideal_age_value = age_avg_btwn[index_of_min]
Ideal_age_gini = (Ideal_age_value, min(Gini_age))

Ideal_age_gini

(89.0, 0.3682252817958181)

### Formalizing the process

Now that we have gone through the process for one category, we want to formalize the process for multiple categories, so we can find the best starting question for our decision tree.

#### For numerical categories

In [43]:
def NumericalGiniImpurity(variable_array, variable_outcome_array, OutcomePlus, OutcomeNeg):
    '''
    This function takes in a variable array and a 2d variable outcome array and returns the gini impurity for that variable based on that outcome.

    Parameters
    -------------------
    Variable array: 1D array
        A 1-D array that represents the variable for each datapoint in the data set. For example, age, capital gain, etc.

    variable_outcome_array: 2D array
        A 2-D array with elements for each data point. Each element is a 1-D array of length 2, which holds the variable itself (ex: age) and the outcome (ex: salary)
        that is trying to be predicted.

    OutcomePlus/Neg: Any
        Check for the outcomes we are expecting.

    Returns
    ----------------------
    Ideal_variable_gini: Tuple
        A tuple with two elements; the ideal variable to start a decision tree with, and its associated gini impurity.

    '''

    #Let's make our data into numpy arrays, for ease of computation
    variable_array = np.array(variable_array)
    variable_outcome_array = np.array(variable_outcome_array)


    #First, we order the data numerically, excluding repeat values, and find the mean value between each value in the array

    var_sorted = np.unique(np.sort(variable_array))

    var_avg_btwn = np.zeros(len(var_sorted) - 1) #1 element shorter b/c taking avg values in between each value in the current array

    for i in range(len(var_sorted) - 1):
        var_avg_btwn[i] = (var_sorted[i] + var_sorted[i+1])/2
    
    # Now we have the values for which we will compute the gini impurity and find the ideal result

    #Here, we initialize variables and arrays we will need

    Gini_var = np.zeros(len(var_avg_btwn))
    datapoints = len(variable_outcome_array)

    LowVar_PlusOutcome = 0
    LowVar_NegOutcome = 0
    HighVar_PlusOutcome = 0
    HighVar_NegOutcome = 0

    # Loop over each avg age
    for count, var_avg_btwn in enumerate(var_avg_btwn):

        # Find the number of people in each category
        for j in variable_outcome_array:
            if j[0] < var_avg_btwn:
                if j[1] == OutcomePlus:
                    LowVar_PlusOutcome += 1
                else:
                    LowVar_NegOutcome += 1
            if j[0] > var_avg_btwn:
                if j[1] == OutcomePlus:
                    HighVar_PlusOutcome += 1
                else:
                    HighVar_NegOutcome += 1

        #Compute the gini impurity for each leaf

        gini_low_var = 1 - (LowVar_PlusOutcome/datapoints)**2 - (LowVar_NegOutcome/datapoints)**2
        gini_high_var = 1 - (HighVar_PlusOutcome/datapoints)**2 - (HighVar_NegOutcome/datapoints)**2

        #Compute the total gini for this age by weighing the gini impurity 

        total_gini = ((LowVar_NegOutcome + LowVar_PlusOutcome)/datapoints)*gini_low_var +((HighVar_NegOutcome + HighVar_PlusOutcome)/datapoints)*gini_high_var

        Gini_var[count] = total_gini

        LowVar_PlusOutcome = 0
        LowVar_NegOutcome = 0
        HighVar_PlusOutcome = 0
        HighVar_NegOutcome = 0

    #Redefine var_avg_btwn b/c it was renamed during enumerate
    var_avg_btwn = np.zeros(len(var_sorted) - 1) #1 element shorter b/c taking avg values in between each value in the current array
    for i in range(len(var_sorted) - 1):
        var_avg_btwn[i] = (var_sorted[i] + var_sorted[i+1])/2


    # Now make the tuple
    index_of_min = np.argmin(Gini_var)
    Ideal_var_value = var_avg_btwn[index_of_min]
    Ideal_var_gini = (Ideal_var_value, min(Gini_var))

    return Ideal_var_gini
    

#### Test function on age data

In [44]:
OutcomePlus = '>50K'
OutcomeNeg = '<=50K'

NumericalGiniImpurity(adults['Age'], adults[['Age','Salary']], OutcomePlus, OutcomeNeg)

(89.0, 0.3682252817958181)

It worked! Now we can compute the gini impurity for all of our numerical parameters

In [47]:
NumericalGinis = [] #Number of numeric variables

NumericalGinis.append(NumericalGiniImpurity(adults['Age'], adults[['Age', 'Salary']], OutcomePlus, OutcomeNeg))
NumericalGinis.append(NumericalGiniImpurity(adults['EdNum'], adults[['EdNum', 'Salary']], OutcomePlus, OutcomeNeg))
NumericalGinis.append(NumericalGiniImpurity(adults['HoursPerWeek'], adults[['HoursPerWeek', 'Salary']], OutcomePlus, OutcomeNeg))

In [48]:
NumericalGinis

[(89.0, 0.3682252817958181),
 (1.5, 0.3690062683214883),
 (1.5, 0.3668983860809549)]

### For categorical data

For categorical data, since we have more than one option per column, we will look at each option in each column and find the option with the lowest gini impurity. That option will be representative of the column as a whole!

We can modify the above function slightly to fit the bill

In [1]:
def CategoricalGiniImpurity(variable_array, variable_outcome_array, OutcomePlus, OutcomeNeg):
    '''
    This function takes in a variable array and a 2d variable outcome array and returns the gini impurity for that variable based on that outcome.

    Parameters
    -------------------
    Variable array: 1D array
        A 1-D array that represents the variable for each datapoint in the data set. For example, age, capital gain, etc.

    variable_outcome_array: 2D array
        A 2-D array with elements for each data point. Each element is a 1-D array of length 2, which holds the variable itself (ex: age) and the outcome (ex: salary)
        that is trying to be predicted.

    OutcomePlus/Neg: Any
        Check for the outcomes we are expecting.

    Returns
    ----------------------
    Ideal_variable_gini: Tuple
        A tuple with two elements; the ideal variable to start a decision tree with, and its associated gini impurity.

    '''

    #Let's make our data into numpy arrays, for ease of computation
    variable_array = np.array(variable_array)
    variable_outcome_array = np.array(variable_outcome_array)


    #First, we order the data numerically, excluding repeat values, and find the mean value between each value in the array

    var_sorted = np.unique(variable_array)
    
    # Now we have the values for which we will compute the gini impurity and find the ideal result

    #Here, we initialize variables and arrays we will need

    Gini_var = np.zeros(len(var_sorted))
    datapoints = len(variable_outcome_array)

    LowVar_PlusOutcome = 0
    LowVar_NegOutcome = 0
    HighVar_PlusOutcome = 0
    HighVar_NegOutcome = 0

    # Loop over each avg age
    for count, var_sorted in enumerate(var_sorted):

        # Find the number of people in each category
        for j in variable_outcome_array:
            if j[0] == var_sorted:
                if j[1] == OutcomePlus:
                    LowVar_PlusOutcome += 1
                else:
                    LowVar_NegOutcome += 1
            if j[0] != var_sorted:
                if j[1] == OutcomePlus:
                    HighVar_PlusOutcome += 1
                else:
                    HighVar_NegOutcome += 1

        #Compute the gini impurity for each leaf

        gini_low_var = 1 - (LowVar_PlusOutcome/datapoints)**2 - (LowVar_NegOutcome/datapoints)**2
        gini_high_var = 1 - (HighVar_PlusOutcome/datapoints)**2 - (HighVar_NegOutcome/datapoints)**2

        #Compute the total gini for this age by weighing the gini impurity 

        total_gini = ((LowVar_NegOutcome + LowVar_PlusOutcome)/datapoints)*gini_low_var +((HighVar_NegOutcome + HighVar_PlusOutcome)/datapoints)*gini_high_var

        Gini_var[count] = total_gini

        LowVar_PlusOutcome = 0
        LowVar_NegOutcome = 0
        HighVar_PlusOutcome = 0
        HighVar_NegOutcome = 0

    #Redefine var_sorted b/c it was renamed during enumerate
    var_sorted = np.unique(variable_array)


    # Now make the tuple
    index_of_min = np.argmin(Gini_var)
    Ideal_var_value = var_sorted[index_of_min]
    Ideal_var_gini = (Ideal_var_value, min(Gini_var))

    return Ideal_var_gini
    

Now we run the function on all the categorical variables!

In [61]:
CategoricalGinis = []
CategoricalGinis.append(CategoricalGiniImpurity(adults['workclass'], adults[['workclass','Salary']], OutcomePlus, OutcomeNeg))
CategoricalGinis.append(CategoricalGiniImpurity(adults['Education'], adults[['Education','Salary']], OutcomePlus, OutcomeNeg))
CategoricalGinis.append(CategoricalGiniImpurity(adults['MaritalStatus'], adults[['MaritalStatus','Salary']], OutcomePlus, OutcomeNeg))
CategoricalGinis.append(CategoricalGiniImpurity(adults['Occupation'], adults[['Occupation','Salary']], OutcomePlus, OutcomeNeg))
CategoricalGinis.append(CategoricalGiniImpurity(adults['Relationship'], adults[['Relationship','Salary']], OutcomePlus, OutcomeNeg))
CategoricalGinis.append(CategoricalGiniImpurity(adults['Race'], adults[['Race','Salary']], OutcomePlus, OutcomeNeg))
CategoricalGinis.append(CategoricalGiniImpurity(adults['Sex'], adults[['Sex','Salary']], OutcomePlus, OutcomeNeg))
CategoricalGinis.append(CategoricalGiniImpurity(adults['NativeCountry'], adults[['NativeCountry','Salary']], OutcomePlus, OutcomeNeg))

Now let's combine the gini lists and see which gini value is the lowest. That will be our starting question!

In [62]:
CategoricalGinis.append(NumericalGinis)
Ginis = CategoricalGinis
Ginis

[(' Never-worked', 0.3661033110805593),
 (' Preschool', 0.3690062683214883),
 (' Married-AF-spouse', 0.36684206019387566),
 (' Armed-Forces', 0.3662036465033548),
 (' Other-relative', 0.42713587693831256),
 (' Other', 0.38260530621723887),
 (' Female', 0.7983926929107752),
 (' Holand-Netherlands', 0.36570674067270975),
 [(89.0, 0.3682252817958181),
  (1.5, 0.3690062683214883),
  (1.5, 0.3668983860809549)]]

From the cell above, we can see that "workclass" has the lowest gini impurity, with the best category being 'Never-worked.' That question will start our decision tree. But where do we go from there?

## Building the complete decision tree